In [8]:
import numpy as np
import pandas as pd
# Programa para métricas de avaliação por matriz de confusão de um classificador
# VP/TP -> VERDADEIRO POSITIVO/TRUE POSITIVE
# FP -> FALSO POSITIVO/FALSE POSITIVE
# VN/TN -> VERDADEIRO NEGATIVO/TRUE NEGATIVE
# FN -> FALSO NEGATIVO/FALSE NEGATIVE
# N -> TOTAL DE AMOSTRAS
# Accuracy/Acurácia: (VP+VN)/N
# Sensitivity/Sensibilidade ou Recall/Revocação: VP/(FN+VP) (True Positive Rate/Taxa de Verdadeiro Positivo)
# Specitivity/Especificidade: VN/(FP+VN) (True Negative Rate/Taxa de Verdadeiro Negativo)
# Precision/Precisão: VP/(VP+FP)
# F-Score: 2*(Precision*Recall/(Precision+Recall))

In [25]:
class MetricsFromCM:
    """
    Cálculo de métricas de avaliação de um modelo de inferência através de uma matriz de confusão. A entrada deve ser um numpy array NxN (e.g. 3x3).
    Obtém o número de verdadeiros positivo e negativo e falsos positivo e negativo para cada classe e calcula:
    - Acurácia
    - Sensibilidade
    - Especificidade
    - Precisão
    - F1-score
    Por fim, calcula as métricas globais do modelo:
    - Micro F1-score
    - Macro F1-score
    - Macro F1-score com pesos para cada classe inferida
    """
    def __init__(self, cm):
        if hasattr(cm, "shape"):
            assert len(cm.shape) == 2 and (cm.shape[0] == cm.shape[1]), 'Invalid Confusion Matrix - (N x N) numpy array is expected'
            self.ConfusionMatrix = cm 
            self.ClassNumber = cm.shape[0]
            self.classInferred = []
            self.classMetrics = []
            for i in range(self.ClassNumber):
                Samples = np.sum(self.ConfusionMatrix[:, i])
                TP = self.ConfusionMatrix[i][i]
                FP = np.sum(np.delete(self.ConfusionMatrix[:, i], i))
                TN = np.sum(np.delete(np.delete(self.ConfusionMatrix, i, 0), i, 1).flatten())
                FN = np.sum(np.delete(self.ConfusionMatrix[i, :], i))
                Accuracy = (TP+TN)/(TP+TN+FP+FN)
                Specitivity = TN/(FP+TN)
                Precision = TP/(TP+FP)
                Recall = TP/(FN+TP)
                Fscore = 2*(Precision*Recall)/(Precision+Recall)
                self.classInferred.append({'Real Samples': Samples, 'TP': TP, 'FP': FP, 'TN': TN, 'FN': FN})
                self.classMetrics.append({'Accuracy': Accuracy, 'Specitivity': Specitivity, 'Precision': Precision, 'Recall': Recall, 'F1-Score': Fscore})
            
            self.Globals = {'Micro F1-Score': self.MicroF1(), 'Macro F1-Score': self.MacroF1(), 'Macro Weighted F1-Score': self.WeightedF1()}
            print('Confusion Matrix:')
            print(self.ConfusionMatrix)
            print('Inferences: {} samples'.format(np.sum(self.ConfusionMatrix.flatten())))
            print('Classification: {} classes'.format(self.ClassNumber))
            print('===========================================================')
            print('Global Scores:')
            globals = pd.DataFrame(self.Globals.values(), self.Globals.keys()).T.to_string(index=False)
            print(globals)
            print('===========================================================')
            print('Class Metrics:')
            print(pd.DataFrame(self.classMetrics))
            
            
        else:
            print('Invalid Input')  

    def MicroF1(self) -> float:
        TP = []
        FP = []
        FN = []
        for item in self.classInferred:
            for key in item.keys():
                if key == 'TP':
                    TP.append(item[key])                    
                if key == 'FP':
                    FP.append(item[key])                        
                if key == 'FN':
                    FN.append(item[key])
        TP = sum(TP)
        FP = sum(FP)
        FN = sum(FN)
        Precision = TP/(TP+FP)
        Recall = TP/(FN+TP)

        return 2*(Precision*Recall)/(Precision+Recall)
    
    def MacroF1(self) -> float:
        Fscores = [item['F1-Score'] for item in self.classMetrics]
        return sum(Fscores)/len(Fscores)
    
    def WeightedF1(self) -> float:
        Samples = [item['Real Samples'] for item in self.classInferred]
        Fscores = [item['F1-Score'] for item in self.classMetrics]
        Weighted = [Samples[i]*Fscores[i] for i in range(self.ClassNumber)]

        return sum(Weighted)/sum(Samples)
     
        

In [26]:
dummy_cm = np.array([[3, 1, 0, 4], [0, 4, 3, 1], [0, 2, 2, 6], [3, 1, 1, 10]])
#dummy_cm = np.array([[10, 1], [3, 7]])
#dummy_cm = np.identity(3)
metrics = MetricsFromCM(dummy_cm)

Confusion Matrix:
[[ 3  1  0  4]
 [ 0  4  3  1]
 [ 0  2  2  6]
 [ 3  1  1 10]]
Inferences: 41 samples
Classification: 4 classes
Global Scores:
 Micro F1-Score  Macro F1-Score  Macro Weighted F1-Score
       0.463415        0.433532                 0.481417
Class Metrics:
   Accuracy  Specitivity  Precision    Recall  F1-Score
0  0.804878     0.909091   0.500000  0.375000  0.428571
1  0.804878     0.878788   0.500000  0.500000  0.500000
2  0.707317     0.870968   0.333333  0.200000  0.250000
3  0.609756     0.576923   0.476190  0.666667  0.555556
